In [45]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from eden.util import configure_logging
import logging
logger = logging.getLogger()
configure_logging(logger,verbosity=0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
import cPickle as pickle
def save_data(data, fname='data.dat'):
    with open(fname, 'wb') as outfile:
        pickle.dump(data, outfile, pickle.HIGHEST_PROTOCOL)
    if len(data.shape)==1:
        print 'Saved %s (%d)' % (fname, data.shape[0])
    else:
        print 'Saved %s (%d,%d)' % (fname, data.shape[0], data.shape[1])

def load_data(fname='data.dat'):
    with open(fname, 'rb') as infile:
        data = pickle.load(infile)
    if len(data.shape)==1:
        print 'Loaded %s (%d)' % (fname, data.shape[0])
    else:
        print 'Loaded %s (%d,%d)' % (fname, data.shape[0], data.shape[1])
    return data

#Predicitve model performance evaluation

In [47]:
def train_perform(X_train, y_train, X_test,y_test):
    # Induce a predictive model
    print 'Training on data matrix [%d x %d]' %(X_train.shape)
    from sklearn.linear_model import SGDClassifier
    estimator = SGDClassifier(average=True, class_weight='auto', shuffle=True, n_jobs=-1)
    estimator.fit(X_train,y_train)

    # Print predictive performance
    from sklearn.metrics import confusion_matrix
    print 'Confusion matrix:'
    print(confusion_matrix(y_test, estimator.predict(X_test)))
    print
    from sklearn.metrics import classification_report
    print 'Classification Report:'
    print classification_report(y_test, estimator.predict(X_test))

#Data preparation

Split data into train/test according to binary clustering

In [48]:
def train_test_ids_split(X, confidence_threshold=1):
    from sklearn.cluster import MiniBatchKMeans
    kmeans = MiniBatchKMeans(n_clusters=2)
    classes = kmeans.fit_predict(X)
    
    from sklearn.linear_model import SGDClassifier
    estimator = SGDClassifier(average=True, class_weight='auto', shuffle=True, n_jobs=-1)
    estimator.fit(X,classes)
    conf = estimator.decision_function(X)
    
    train_ids = []
    test_ids = []
    for i,(class_info, conf_info) in enumerate(zip(classes, conf)):
        if abs(conf_info) > confidence_threshold:
            if class_info == 0:
                train_ids.append(i)
            else:
                test_ids.append(i)
    return train_ids, test_ids

Retrieve seed sequences from RFam families.

Use EDeN to transform sequences to vectors. Use RNAfold to create structures and EDeN to convert those to vectors.

Use the sequence vectors to guide the train/test split. 

In [49]:
from sklearn.random_projection import SparseRandomProjection
import time

def rfam_uri(family_id):
        return 'http://rfam.xfam.org/family/%s/alignment?acc=%s&format=fastau&download=0'%(family_id,family_id)

#RNAfold
def pre_processor(seqs):
    from eden.converter.rna.rnafold import rnafold_to_eden
    graphs = rnafold_to_eden(seqs)
    from eden.modifier.graph import structure 
    graphs = structure.basepair_to_nesting(graphs)
    return graphs

    
#RNAVectorizer
def pre_processor(seqs):
    n_neighbors = min(len(seqs),30)
    rs = int(time.time())
    from eden.RNA import Vectorizer as RNAVectorizer
    rnavec=RNAVectorizer(n_neighbors=n_neighbors,
                         sampling_prob=.15,
                         n_iter=5,
                         min_energy=-5,
                         random_state=rs)
    rnavec.fit(seqs)
    graphs = rnavec.graphs(seqs)
    from eden.modifier.graph import structure 
    graphs = structure.basepair_to_nesting(graphs)
    return graphs


def rfam_to_matrix(rfam_id, use_structure=True, n_max=50, complexity=2, nbits=10):
    from eden.converter.fasta import fasta_to_sequence
    seqs = fasta_to_sequence(rfam_uri(rfam_id))
    from itertools import islice
    seqs = islice(seqs,n_max)
    seqs = list(seqs)
    if use_structure:
        graphs = pre_processor(seqs)
        from eden.graph import Vectorizer as GraphVectorizer
        vectorizer = GraphVectorizer(r=3,d=0,min_r=3,
                                     normalization=False,
                                     inner_normalization=False,
                                     nbits=nbits)
        X = vectorizer.transform(graphs)
    else:
        from eden.path import Vectorizer as SeqVectorizer
        vectorizer = SeqVectorizer(r=3,d=20,min_r=3,
                                   normalization=False,
                                   inner_normalization=False,
                                   nbits=nbits)
        X = vectorizer.transform(seqs)
    return X

def rfam_data(rfam_ids, n_max=300, complexity=3, nbits=13):
    import numpy as np
    from scipy.sparse import vstack
    seq_train_list = []
    seq_test_list = []
    struct_train_list = []
    struct_test_list = []
    y_train = []
    y_test = []
    for i,rfam_id in enumerate(rfam_ids):
        # seq case
        seq_X=rfam_to_matrix(rfam_id, use_structure=False, n_max=n_max, complexity=complexity, nbits=nbits)
        train_ids, test_ids = train_test_ids_split(seq_X)
        seq_X_train = seq_X[train_ids]
        seq_X_test = seq_X[test_ids]
        seq_train_list.append(seq_X_train)
        seq_test_list.append(seq_X_test)
        y_train += [i] * seq_X_train.shape[0]
        y_test += [i] * seq_X_test.shape[0]
        
        # struct case
        struct_X=rfam_to_matrix(rfam_id, use_structure=True, n_max=n_max, complexity=complexity, nbits=nbits)
        #NOTE: use the same split given by the sequence similarity
        struct_X_train = struct_X[train_ids]
        struct_X_test = struct_X[test_ids]
        struct_train_list.append(struct_X_train)
        struct_test_list.append(struct_X_test)
    seq_X_train = vstack(seq_train_list, format="csr")
    seq_X_test = vstack(seq_test_list, format="csr")
    struct_X_train = vstack(struct_train_list, format="csr")
    struct_X_test = vstack(struct_test_list, format="csr")
    target_train = np.array(y_train)
    target_test = np.array(y_test)
    
    return seq_X_train,\
        seq_X_test,\
        struct_X_train,\
        struct_X_test,\
        target_train,\
        target_test

rfam_ids=['RF00004','RF00005','RF00015','RF00020','RF00026','RF00169',
          'RF00380','RF00386','RF01051','RF01055','RF01234','RF01699',
          'RF01701','RF01705','RF01731','RF01734','RF01745','RF01750',
          'RF01942','RF01998','RF02005','RF02012','RF02034']

print 'num families:', len(rfam_ids)

num families: 23


In [50]:
data_ids=rfam_ids[0:20]
print 'Experiment using %d families' % len(data_ids)
prefix='f3_c4nb12_'

Experiment using 20 families


In [51]:
%%time
seq_X_train,\
seq_X_test,\
struct_X_train,\
struct_X_test,\
y_train,\
y_test = rfam_data(data_ids, n_max=300, complexity=4, nbits=11)

CPU times: user 5min 34s, sys: 4min 4s, total: 9min 38s
Wall time: 28min 56s


In [52]:
%%time
save_data(seq_X_train, fname=prefix+'seq_X_train.dat')
save_data(seq_X_test, fname=prefix+'seq_X_test.dat')
save_data(struct_X_train, fname=prefix+'struct_X_train.dat')
save_data(struct_X_test, fname=prefix+'struct_X_test.dat')
save_data(y_train, fname=prefix+'y_train.dat')
save_data(y_test, fname=prefix+'y_test.dat')

Saved f3_c4nb12_seq_X_train.dat (1811,2049)
Saved f3_c4nb12_seq_X_test.dat (2224,2049)
Saved f3_c4nb12_struct_X_train.dat (1811,2049)
Saved f3_c4nb12_struct_X_test.dat (2224,2049)
Saved f3_c4nb12_y_train.dat (1811)
Saved f3_c4nb12_y_test.dat (2224)
CPU times: user 24.1 ms, sys: 134 ms, total: 158 ms
Wall time: 483 ms


In [53]:
%%time
seq_X_train=load_data(fname=prefix+'seq_X_train.dat')
seq_X_test=load_data(fname=prefix+'seq_X_test.dat')
struct_X_train=load_data(fname=prefix+'struct_X_train.dat')
struct_X_test=load_data(fname=prefix+'struct_X_test.dat')
y_train=load_data(fname=prefix+'y_train.dat')
y_test=load_data(fname=prefix+'y_test.dat')

Loaded f3_c4nb12_seq_X_train.dat (1811,2049)
Loaded f3_c4nb12_seq_X_test.dat (2224,2049)
Loaded f3_c4nb12_struct_X_train.dat (1811,2049)
Loaded f3_c4nb12_struct_X_test.dat (2224,2049)
Loaded f3_c4nb12_y_train.dat (1811)
Loaded f3_c4nb12_y_test.dat (2224)
CPU times: user 27.2 ms, sys: 56.5 ms, total: 83.7 ms
Wall time: 82.9 ms


In [54]:
%%time
# sparse to dense arrays
Xseq_train = seq_X_train
Xseq_test = seq_X_test
X_train = struct_X_train
X_test = struct_X_test

CPU times: user 67 µs, sys: 7.02 ms, total: 7.09 ms
Wall time: 7.1 ms


In [55]:
%%time
# sparse to dense arrays
Xseq_train = seq_X_train.toarray()
Xseq_test = seq_X_test.toarray()
X_train = struct_X_train.toarray()
X_test = struct_X_test.toarray()

CPU times: user 60.5 ms, sys: 44.1 ms, total: 105 ms
Wall time: 141 ms


#Sequence features

In [56]:
%%time
train_perform(Xseq_train, y_train, Xseq_test,y_test)

Training on data matrix [1811 x 2049]
Confusion matrix:
[[ 26   0   0   0   0   0   0   6   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0  13   1   1   1   0   4  59   0   1   1   1   3   1   1   0   0   0
    0   0]
 [  0   0   1   0   0   0   0 108   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0  34   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0  26   0   1 128   0   0   0   2   0   0   0   0   0   0
    0   0]
 [  0  10   0   0   0  17   0 172   0   2   0   0  20   1   1   0   0   0
    0   0]
 [  0   0   0   0   0   0  14 110   1   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0  68   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   1   0   0   0   0   2 112   8   0   0   2   4   0   2   0   0   0
    0   0]
 [  0   0   0   0   0   0   0 108   0   6   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0  40   0   0  64   0   0   0   0   0   0   0
    0   0]
 [  0   0

#Structure features

In [57]:
%%time
train_perform(X_train, y_train, X_test,y_test)

Training on data matrix [1811 x 2049]
Confusion matrix:
[[32  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 3 31  4  9  9  0  2  0  0  1  0  1  9  5  5  1  1  0  6  0]
 [ 4 23  9  4  0  0  0 29  0  1  0  8  5  1 20  0  2  0  3  0]
 [ 0  0  0 34  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2 39  0  0 95  0  4  0  2  0  1  2  4  1  4  0  0  0  3  0]
 [12 46  1 12  2 27  4 10  0  0  9  4 52  6  4  0  1  3 26  4]
 [ 0 16  0  0  1  5 39  1  0  0  0 10  9  1 30  0  2  0 11  0]
 [ 3  1  0  0  0  1  0 61  0  0  0  0  1  0  0  0  0  0  1  0]
 [ 0  9  1  1 15  2  7  0  7  1  6 38 20  7  4  0  2  2  8  1]
 [ 2 34  1 12  2  2  0  7  0  1  2  2  4  5 27  0  0  1  9  3]
 [ 0  0  0  0  0  0  0  1  0  0 97  0  1  0  0  0  0  0  5  0]
 [ 0  0  0  0  0  0  3  0  0  0  0 45  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 85  0  0  0  0  0  0  0]
 [ 3  9  0  0  1  0  0  0  0  3  0  0  2 96  0  0  1  1  4  0]
 [ 0  0  0  1  0  2  0  2  0  0  0  0  1  0  6  0  0  0  0  0]

#Learned map seq $\mapsto$ structure features

In [58]:
from sklearn.preprocessing import StandardScaler
seqs_scale = StandardScaler(with_mean=True)
data_matrix_in = seqs_scale.fit_transform(Xseq_train)
struct_scale = StandardScaler(with_mean=True)
data_matrix_out = struct_scale.fit_transform(X_train)
n_features_in = data_matrix_in.shape[1]
n_features_out = data_matrix_out.shape[1]
n_features_hidden = max(n_features_in, n_features_out) * 2
print 'n_neurons: #in [%d] -- #hidden [%d] -- #out [%d]' % (n_features_in, n_features_hidden, n_features_out)

n_neurons: #in [2049] -- #hidden [4098] -- #out [2049]


In [59]:
from sknn.mlp import Regressor, Layer

net = Regressor(layers=[
        Layer("Rectifier", units=n_features_hidden),
        Layer("Rectifier", units=n_features_hidden),
        Layer("Rectifier", units=n_features_hidden),
        Layer("Softmax", units=n_features_out)],
                learning_rate=0.000001,
                n_iter=100,
                regularize='L1',
                valid_size=0.2)

In [60]:
%%time
net.fit(data_matrix_in, data_matrix_out)


A runtime exception was caught during training. This likely occurred due to
a divergence of the SGD algorithm, and NaN floats were found by PyLearn2.

Try setting the `learning_rate` 10x lower to resolve this, for example:
    learning_rate=0.000000



RuntimeError: NaN in hidden0_W

In [ ]:
%%time
# Transform seq features to struct features
X_train_pred = net.predict(seqs_scale.transform(Xseq_train))
X_test_pred = net.predict(seqs_scale.transform(Xseq_test))

In [ ]:
%%time
train_perform(X_train_pred, y_train, X_test_pred, y_test)

---